Question 1 (80 points)

In this question you are tasked with writing modified versions of the web crawler that we covered in class. More specifically, you asked to create two webcrawlers for the following tasks:

1.Crawl pages whose seed url is the press releases page of the Federal Reserve System: https://www.federalreserve.gov/newsevents/pressreleases.htm and collect pages that contain the word "covid" found within the page. The goal is to collect at least 10 such urls. At the end of the crawling the code should output the urls of the webpages found to contain the word "covid". When checking whether the word is present on the webpage you should consider lower- and upper-case word versions (Covid, COVID, covid). One way to do this is to lowercase the webpage text prior to doing word matching.

In [3]:
import urllib.request
from bs4 import BeautifulSoup
import requests

In [4]:
# Crawl 10 Pages with word "covid" and store in List
seed_url = "https://www.federalreserve.gov/newsevents/pressreleases.htm"
domain = "https://www.federalreserve.gov/newsevents/pressreleases"

urls = [seed_url]  # queue of urls to crawl
seen = [seed_url]  # stack of urls seen so far
opened = []
covidurl = []

maxNumUrl = 10;  # set the maximum number of urls included 'covid' to visit
print("Starting with url=" + str(urls))
while len(urls) > 0 and len(covidurl) < maxNumUrl:
    # DEQUEUE A URL FROM urls AND TRY TO OPEN AND READ IT
    try:
        curr_url = urls.pop(0)  # pop：delete the first item of the list
        print("num. of URLs in stack: %d " % len(urls))
        print("Trying to access= " + curr_url)
        req = urllib.request.Request(curr_url, headers={'User-Agent': 'Mozilla/5.0'}) 
        webpage = urllib.request.urlopen(req).read()
        opened.append(curr_url)

    except Exception as ex:
        print("Unable to access= " + curr_url)
        print(ex)
        continue  # skip code below

    # IF URL OPENS, CHECK WHICH URLS THE PAGE CONTAINS
    # ADD THE URLS FOUND TO THE QUEUE url AND seen
    soup = BeautifulSoup(webpage)  # creates object soup 
    
    # Put child URLs into the stack
    for tag in soup.find_all('a', href=True):  # find tags with links
        childUrl = tag['href']  # extract just the link
        childUrl = urllib.parse.urljoin(seed_url, childUrl)  # join two urls
        if domain in childUrl and childUrl not in seen:
            urls.append(childUrl)
            seen.append(childUrl)
            if 'covid' in requests.get(childUrl).text.lower():  #find childurls with "covid"
                covidurl.append(childUrl)
                if len(covidurl) >= maxNumUrl:
                    break

print("num. of URLs seen = %d, and scanned = %d" % (len(seen), len(opened)))

Starting with url=['https://www.federalreserve.gov/newsevents/pressreleases.htm']
num. of URLs in stack: 0 
Trying to access= https://www.federalreserve.gov/newsevents/pressreleases.htm
num. of URLs in stack: 21 
Trying to access= https://www.federalreserve.gov/newsevents/pressreleases.htm#content
num. of URLs in stack: 20 
Trying to access= https://www.federalreserve.gov/newsevents/pressreleases/2022-press-fomc.htm
num. of URLs in stack: 32 
Trying to access= https://www.federalreserve.gov/newsevents/pressreleases/2022-press.htm
num. of URLs seen = 147, and scanned = 4


In [5]:
# Output 10 links with word "covid" in csv file
import pandas as pd
covid_links = pd.DataFrame(covidurl)
covid_links.columns = ['covid_links']
covid_links.to_csv('covid_links.csv',mode='w+',header=True, index=None)